This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](natural.ipynb)

# Non-Uniform Natural Splines

The derivation is similar to
[the uniform case](natural-uniform.ipynb),
but this time the parameter intervals can have arbitrary values.

In [ ]:
import sympy as sp
sp.init_printing(order='grevlex')

In [ ]:
from utility import NamedExpression

In [ ]:
t = sp.symbols('t')

Just like in the uniform case,
we are considering two adjacent spline segments,
but this time we must allow arbitrary parameter values:

In [ ]:
t3, t4, t5 = sp.symbols('t3:6')

In [ ]:
b_monomial = sp.Matrix([t**3, t**2, t, 1]).T
b_monomial

In [ ]:
coefficients3 = sp.symbols('a:dbm3')[::-1]
coefficients4 = sp.symbols('a:dbm4')[::-1]

In [ ]:
b_monomial.dot(coefficients3)

In [ ]:
p3 = NamedExpression(
    'pbm3',
    b_monomial.dot(coefficients3).subs(t, (t - t3)/(t4 - t3)))
p4 = NamedExpression(
    'pbm4',
    b_monomial.dot(coefficients4).subs(t, (t - t4)/(t5 - t4)))
display(p3, p4)

In [ ]:
pd3 = p3.diff(t)
pd4 = p4.diff(t)
display(pd3, pd4)

In [ ]:
equations = [
    p3.evaluated_at(t, t3).with_name('xbm3'),
    p3.evaluated_at(t, t4).with_name('xbm4'),
    p4.evaluated_at(t, t4).with_name('xbm4'),
    p4.evaluated_at(t, t5).with_name('xbm5'),
    pd3.evaluated_at(t, t3).with_name('xbmdot3'),
    pd3.evaluated_at(t, t4).with_name('xbmdot4'),
    pd4.evaluated_at(t, t4).with_name('xbmdot4'),
    pd4.evaluated_at(t, t5).with_name('xbmdot5'),
]

We introduce a few new symbols to simplify the display,
but we keep calculating with $t_i$:

In [ ]:
deltas = {
    t3: 0,
    t4: sp.Symbol('Delta3'),
    t5: sp.Symbol('Delta3') + sp.Symbol('Delta4'),
}

In [ ]:
for e in equations:
    display(e.subs(deltas))

In [ ]:
coefficients = sp.solve(equations, coefficients3 + coefficients4)

In [ ]:
for c, e in coefficients.items():
    display(NamedExpression(c, e.subs(deltas)))

In [ ]:
pdd3 = pd3.diff(t)
pdd4 = pd4.diff(t)
display(pdd3, pdd4)

In [ ]:
sp.Eq(pdd3.expr.subs(t, t4), pdd4.expr.subs(t, t4))

In [ ]:
_.subs(coefficients).subs(deltas).simplify()

Like in the uniform case,
we can generalize by renaming index $4$ to $i$:

\begin{equation*}
\frac{1}{\Delta_{i-1}} \dot{\boldsymbol{x}}_{i-1}
+
\left(\frac{2}{\Delta_{i-1}} + \frac{2}{\Delta_i}\right)
\dot{\boldsymbol{x}}_i
+
\frac{1}{\Delta_i} \dot{\boldsymbol{x}}_{i+1}
=
\frac{3 (\boldsymbol{x}_i - \boldsymbol{x}_{i-1})}{{\Delta_{i-1}}^2} +
\frac{3 (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)}{{\Delta_i}^2}
\end{equation*}

We are not showing the full matrix here,
because it would be quite a bit more complicated and less helpful than
in the uniform case.

## End Conditions

Like in the [uniform case](natural-uniform.ipynb#End-Conditions),
we can come up with a few end conditions
in order to define the missing matrix rows.

The Python class [splines.Natural](../python-module/splines.rst#splines.Natural)
uses "natural" end conditions by default.

"Natural" end conditions are derived in
[a separate notebook](end-conditions-natural.ipynb),
yielding these expressions:

\begin{align*}
2 \Delta_0 \dot{\boldsymbol{x}}_0
+
\Delta_0 \dot{\boldsymbol{x}}_1
&=
3 (\boldsymbol{x}_1 - \boldsymbol{x}_0)
\\
\Delta_{N-2} \dot{\boldsymbol{x}}_{N-2}
+
2 \Delta_{N-2} \dot{\boldsymbol{x}}_{N-1}
&=
3 (\boldsymbol{x}_{N-1} - \boldsymbol{x}_{N-2})
\end{align*}

Other end conditions can be derived as shown in
[the notebook about uniform "natural" splines](natural-uniform.ipynb#End-Conditions).